# PointCNN Experiments

## Package Installation, Imports, Function Definitions

In [1]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

     |████████████████████████████████| 10.4 MB 2.4 MB/s 
     |████████████████████████████████| 3.7 MB 2.8 MB/s 
     |████████████████████████████████| 1.6 MB 2.7 MB/s 
     |████████████████████████████████| 870 kB 2.7 MB/s 
     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 407 kB 40.5 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 


In [2]:
import os.path as osp
import time

import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.nn import Linear as Lin
from torch.optim import Adam

from torch_geometric.datasets import ModelNet, ShapeNet
from torch_geometric.loader import DataLoader
from torch_geometric.nn import XConv, fps, global_mean_pool
import torch_geometric.transforms as T
from torch_geometric.utils import intersection_and_union as i_and_u

In [21]:
def get_dataset(name, num_points):
    path = osp.join('data', name)
    pre_transform = T.NormalizeScale()

    if name == 'ModelNet10':
      transform = T.SamplePoints(num_points)

      train_dataset = ModelNet(
          path,
          name='10',
          train=True,
          transform=transform,
          pre_transform=pre_transform)
      test_dataset = ModelNet(
          path,
          name='10',
          train=False,
          transform=transform,
          pre_transform=pre_transform)
    elif name == 'ModelNet40':     
      transform = T.SamplePoints(num_points)

      train_dataset = ModelNet(
          path,
          name='40',
          train=True,
          transform=transform,
          pre_transform=pre_transform)
      test_dataset = ModelNet(
          path,
          name='40',
          train=False,
          transform=transform,
          pre_transform=pre_transform)
    elif name == 'ShapeNet':
      category = None  # Pass in `None` to train on all categories.
      #transform = T.SamplePoints(num_points)
      transform = T.Compose([
          T.RandomTranslate(0.01),
          T.RandomRotate(15, axis=0),
          T.RandomRotate(15, axis=1),
          T.RandomRotate(15, axis=2)
      ])

      train_dataset = ShapeNet(
          path,
          category,
          split='trainval',
          transform=transform,
          pre_transform=pre_transform)
      test_dataset = ShapeNet(
          path,
          category,
          split='test',
          transform=transform,
          pre_transform=pre_transform)

    return train_dataset, test_dataset

## Classification


In [8]:
MODELNET_VERSION = 'ModelNet40' # ModelNet10 or ModelNet40
NUM_POINTS = [512, 768, 1024]

EPOCHS = 100 #200
BATCH_SIZE = 32
LR = 0.001
LR_DECAY_FACTOR = 0.5
LR_DECAY_STEP_SIZE = 50
WEIGHT_DECAY = 0

In [5]:
def train(model, optimizer, train_loader, device):
    model.train()
    losses = []

    for data in train_loader:
        optimizer.zero_grad()
        data = data.to(device)
        out = model(data.pos, data.batch)
        loss = F.nll_loss(out, data.y)
        losses.append(loss)
        loss.backward()
        optimizer.step()

    return sum(losses)/len(losses)

@torch.no_grad()
def test(model, test_loader, device):
    model.eval()

    losses = []
    correct = 0
    for data in test_loader:
        data = data.to(device)
        out = model(data.pos, data.batch)
        loss = F.nll_loss(out, data.y)
        losses.append(loss)
        pred = model(data.pos, data.batch).max(1)[1]
        correct += pred.eq(data.y).sum().item()
    test_acc = correct / len(test_loader.dataset)

    return test_acc, sum(losses)/len(losses)


def run(train_dataset, test_dataset, model, epochs, batch_size, lr,
        lr_decay_factor, lr_decay_step_size, weight_decay):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    duration = 0
    train_losses = []
    test_losses = []
    test_accs = []
    model = model.to(device)
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

    for epoch in range(1, epochs + 1):
        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t_start = time.perf_counter()

        train_loss = train(model, optimizer, train_loader, device)
        train_losses.append(train_loss.item())
        test_acc, test_loss = test(model, test_loader, device)
        #test_loss = test(model, test_loader, device)
        test_accs.append(test_acc)
        test_losses.append(test_loss.item())

        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t_end = time.perf_counter()
        duration += t_end - t_start
        print('Epoch: {:03d}, Test: {:.4f}, Duration: {:.2f}'.format(
            epoch, test_acc, t_end - t_start))

        if epoch % lr_decay_step_size == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_decay_factor * param_group['lr']

    return duration, train_losses, test_losses, test_accs

In [27]:
class Net(torch.nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()

        self.conv1 = XConv(0, 48, dim=3, kernel_size=8, hidden_channels=32)
        self.conv2 = XConv(48, 96, dim=3, kernel_size=12, hidden_channels=64, dilation=2)
        self.conv3 = XConv(96, 192, dim=3, kernel_size=16, hidden_channels=128, dilation=2)
        self.conv4 = XConv(192, 384, dim=3, kernel_size=16, hidden_channels=256, dilation=3)

        self.lin1 = Lin(384, 256)
        self.lin2 = Lin(256, 128)
        self.lin3 = Lin(128, num_classes)

    def forward(self, pos, batch):
        x = F.relu(self.conv1(None, pos, batch))

        idx = fps(pos, batch, ratio=0.375)
        x, pos, batch = x[idx], pos[idx], batch[idx]

        x = F.relu(self.conv2(x, pos, batch))

        idx = fps(pos, batch, ratio=0.334)
        x, pos, batch = x[idx], pos[idx], batch[idx]

        x = F.relu(self.conv3(x, pos, batch))
        x = F.relu(self.conv4(x, pos, batch))

        x = global_mean_pool(x, batch)

        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin3(x)
        return F.log_softmax(x, dim=-1)


In [ ]:
results = dict()
for POINTS in NUM_POINTS:
  train_dataset, test_dataset = get_dataset(name=MODELNET_VERSION, num_points=POINTS)
  model = Net(train_dataset.num_classes)

  duration, train_losses, test_losses, test_accs = run(train_dataset, test_dataset, model, EPOCHS, BATCH_SIZE, LR, LR_DECAY_FACTOR, LR_DECAY_STEP_SIZE, WEIGHT_DECAY)

  results[str(POINTS)+'_duration'] = duration
  results[str(POINTS)+'_train_losses'] = train_losses
  results[str(POINTS)+'_test_losses'] = test_losses
  results[str(POINTS)+'_test_accs'] = test_accs

In [ ]:
results.keys()

In [ ]:
plt.plot(range(1,EPOCHS+1), results['512_train_losses'], label='512')
plt.plot(range(1,EPOCHS+1), results['768_train_losses'], label='768')
plt.plot(range(1,EPOCHS+1), results['1024_train_losses'], label='1024')
plt.title(MODELNET_VERSION + ' Train Losses varying Num Points')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(1,EPOCHS+1), results['512_test_losses'], label='512')
plt.plot(range(1,EPOCHS+1), results['768_test_losses'], label='768')
plt.plot(range(1,EPOCHS+1), results['1024_test_losses'], label='1024')
plt.title(MODELNET_VERSION + ' Test Losses varying Num Points')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(range(1,EPOCHS+1), results['512_test_accs'], label='512')
plt.plot(range(1,EPOCHS+1), results['768_test_accs'], label='768')
plt.plot(range(1,EPOCHS+1), results['1024_test_accs'], label='1024')
plt.title(MODELNET_VERSION + ' Test Accuracies varying Num Points')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
print(results['512_duration'], results['768_duration'], results['1024_duration'])


In [ ]:
print(sum(results['512_test_accs'])/len(results['512_test_accs']))

In [ ]:
print(sum(results['768_test_accs'])/len(results['768_test_accs']))

In [ ]:
print(sum(results['1024_test_accs'])/len(results['1024_test_accs']))

## Segmentation

In [30]:
EPOCHS = 5 #200
BATCH_SIZE = 81931 #32
LR = 0.001
LR_DECAY_FACTOR = 0.5
LR_DECAY_STEP_SIZE = 50
WEIGHT_DECAY = 0

In [28]:
def train(model, optimizer, train_loader, device):
    model.train()

    losses = []
    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.pos, data.batch)
        loss = F.nll_loss(out, data.y)
        losses.append(loss)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0

    return sum(losses)/len(losses)


@torch.no_grad()
def test(model, loader, device):
    model.eval()

    accs = []
    losses = []
    y_mask = loader.dataset.y_mask
    ious = [[] for _ in range(len(loader.dataset.categories))]

    for data in loader:
        data = data.to(device)
        out = model(data)
        #pred = model(data).argmax(dim=1)

        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes
        accs.append(correct_nodes / total_nodes)

        loss = F.nll_loss(out, data.y)
        losses.append(loss)

        pred = out.argmax(dim=1)
        i, u = i_and_u(pred, data.y, loader.dataset.num_classes, data.batch)
        iou = i.cpu().to(torch.float) / u.cpu().to(torch.float)
        iou[torch.isnan(iou)] = 1

        # Find and filter the relevant classes for each category.
        for iou, category in zip(iou.unbind(), data.category.unbind()):
            ious[category.item()].append(iou[y_mask[category]])

    # Compute mean IoU.
    ious = [torch.stack(iou).mean(0).mean(0) for iou in ious]
    return torch.tensor(ious).mean().item(), sum(losses)/len(losses), sum(accs)/len(accs)


def run(train_dataset, test_dataset, model, epochs, batch_size, lr,
        lr_decay_factor, lr_decay_step_size, weight_decay):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    duration = 0
    train_losses = []
    test_losses = []
    test_accs = []
    test_ious = []
    model = model.to(device)
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

    for epoch in range(1, epochs + 1):
        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t_start = time.perf_counter()

        train_loss = train(model, optimizer, train_loader, device)
        train_losses.append(train_loss.item())
        iou, test_acc, test_loss = test(model, test_loader, device)
        #test_loss = test(model, test_loader, device)
        test_ious.append(iou)
        test_accs.append(test_acc)
        test_losses.append(test_loss.item())

        if torch.cuda.is_available():
            torch.cuda.synchronize()

        t_end = time.perf_counter()
        duration += t_end - t_start
        print('Epoch: {:03d}, Test: {:.4f}, Duration: {:.2f}'.format(
            epoch, test_acc, t_end - t_start))

        '''if epoch % lr_decay_step_size == 0:
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_decay_factor * param_group['lr']'''

    return duration, train_losses, test_losses, test_accs, test_ious

In [25]:
class Net(torch.nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()

        self.conv1 = XConv(0, 256, dim=3, kernel_size=8, hidden_channels=256)
        self.conv2 = XConv(256, 256, dim=3, kernel_size=12, hidden_channels=256, dilation=2)
        self.conv3 = XConv(256, 512, dim=3, kernel_size=16, hidden_channels=512, dilation=2)
        self.conv4 = XConv(512, 1024, dim=3, kernel_size=16, hidden_channels=1024, dilation=6)
        self.conv5 = XConv(1024, 512, dim=3, kernel_size=16, hidden_channels=512, dilation=6)
        self.conv6 = XConv(512, 256, dim=3, kernel_size=12, hidden_channels=256, dilation=6)
        self.conv7 = XConv(256, 256, dim=3, kernel_size=8, hidden_channels=256, dilation=6)
        self.conv8 = XConv(256, 256, dim=3, kernel_size=8, hidden_channels=256, dilation=4)

        self.lin1 = Lin(256, 256)
        self.lin2 = Lin(256, 256)
        self.lin3 = Lin(256, num_classes)

    def forward(self, pos, batch):
        x1 = F.relu(self.conv1(None, pos, batch))

        idx = fps(pos, batch, ratio=0.375)
        x, pos, batch = x1[idx], pos[idx], batch[idx]

        x2 = F.relu(self.conv2(x, pos, batch))

        idx = fps(pos, batch, ratio=0.334)
        x, pos, batch = x2[idx], pos[idx], batch[idx]

        x3 = F.relu(self.conv3(x, pos, batch))
        x = F.relu(self.conv4(x3, pos, batch))
        x = F.relu(self.conv5(x+x3, pos, batch))
        x = F.relu(self.conv6(x+x2, pos, batch))
        x = F.relu(self.conv7(x+x1, pos, batch))
        x = F.relu(self.conv8(x+x1, pos, batch))

        x = global_mean_pool(x, batch)

        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin3(x)
        return F.log_softmax(x, dim=-1)


In [ ]:
train_dataset, test_dataset = get_dataset(name='ShapeNet', num_points=None)
model = Net(train_dataset.num_classes)

duration, train_losses, test_losses, test_accs, test_ious = run(train_dataset, test_dataset, model, EPOCHS, BATCH_SIZE, LR, LR_DECAY_FACTOR, LR_DECAY_STEP_SIZE, WEIGHT_DECAY)
